In [1]:
# importing Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)# To see all the columns of a dataframe
#pd.set_option('display.max_rows', None)

In [2]:
# Loading reduced feature training set
X_train = pd.read_csv('X_train_final.csv')
y_train = pd.read_csv('y_train.final.csv')

In [3]:
# Loading reduced feature test set
X_test = pd.read_csv('X_test_final.csv')
y_test = pd.read_csv('y_test.final.csv')

## Model_12: Keras Dense Model with Equal Neurons/layer, ELU Activation & RMS PropTuned learning rate.

In [4]:
# Importing Optuna , Tensorflow & Keras
import optuna , tensorflow as tf

In [5]:
# Importing Keras
from tensorflow import keras

In [6]:
# Importing the Sklearn's roc_auc_score module
from sklearn.metrics import roc_auc_score

In [7]:
# Importing train test split from Sklearn to produce validation set
from sklearn.model_selection import train_test_split

In [8]:
# Performing train_validation split with test size = .15 to provide more Training data to Neural Net
X_train_1, X_val, y_train_1, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42,
                                                          stratify=y_train)

In [9]:
# Defining class weight dictionary list for the imbalanced class
cl_weight = [{0:1.0,1:11.38697},{0:1.0,1:12.0},{0:1.0,1:13.0},{0:1.0,1:14.0},{0:1.0,1:15.0},{0:1.0,1:16.0},
             {0:1.0,1:17.0},{0:1.0,1:18.0}]

In [10]:
# Defining best roc_auc score
best_roc_auc = 0

In [11]:
# defining the objective function
def objective_wrappper_keras_eq_ELU(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        model.add(keras.layers.BatchNormalization())# For normalization
        for i in range(trial.suggest_int('number',2,6)):
            model.add(keras.layers.Dense(units=no_units,activation='elu',
                                     kernel_initializer='he_normal'))
            model.add(keras.layers.BatchNormalization())# Keras layer for normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Elu_eq_Learn.h5')
        
        return roc_auc_1
    
    return objective

In [12]:
# Defining the evaluation function for study's best parameters
def study_best_score_params(X_tr, y_tr, X_v, y_v, obj_func, cl_w, n_trials=100):
    ''' Computes the best hyper parameters of the Neural net using the Training set(X_tr,y_tr) & Validation set
    (X_v,y_v), class weights cl_w & 
    returns Optuna's study's best score & clasifier parameters'''
    
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func(X_tr, y_tr, X_v, y_v, cl_w),n_trials)
    best_score = study.best_value
    best_params = study.best_params
    return (best_score,best_params)

In [13]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq_ELU, cl_weight,
                                                            n_trials=50)

[I 2021-01-05 17:31:49,198] A new study created in memory with name: no-name-39a97cbb-52fe-4f27-80ae-9109cc5b7f50


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 34s 174us/sample - loss: 1.3700 - auc: 0.7060 - val_loss: 4140.6598 - val_auc: 0.7319
Epoch 2/50
196038/196038 [==============================] - 32s 165us/sample - loss: 1.3278 - auc: 0.7279 - val_loss: 1931764.7701 - val_auc: 0.7062
Epoch 3/50
196038/196038 [==============================] - 32s 165us/sample - loss: 1.3154 - auc: 0.7346 - val_loss: 3108750512.0442 - val_auc: 0.7042
Epoch 4/50
196038/196038 [==============================] - 33s 166us/sample - loss: 1.3118 - auc: 0.7359 - val_loss: 5366291157.3223 - val_auc: 0.6943
Epoch 5/50
196038/196038 [==============================] - 32s 165us/sample - loss: 1.3136 - auc: 0.7351 - val_loss: 801698.2132 - val_auc: 0.6862


[I 2021-01-05 17:34:37,101] Trial 0 finished with value: 0.7318557327041448 and parameters: {'step': 6, 'no._units': 166, 'number': 6, 'best_weight': {0: 1.0, 1: 16.0}}. Best is trial 0 with value: 0.7318557327041448.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 35s 179us/sample - loss: 1.1928 - auc_1: 0.7063 - val_loss: 1.2089 - val_auc_1: 0.7294
Epoch 2/50
196038/196038 [==============================] - 33s 168us/sample - loss: 1.1546 - auc_1: 0.7279 - val_loss: 53398.8370 - val_auc_1: 0.7200
Epoch 3/50
196038/196038 [==============================] - 33s 168us/sample - loss: 1.1436 - auc_1: 0.7347 - val_loss: 6193573.9642 - val_auc_1: 0.6905
Epoch 4/50
196038/196038 [==============================] - 33s 167us/sample - loss: 1.1355 - auc_1: 0.7396 - val_loss: 262056.0696 - val_auc_1: 0.6803
Epoch 5/50
196038/196038 [==============================] - 33s 169us/sample - loss: 1.1355 - auc_1: 0.7398 - val_loss: 12045.8039 - val_auc_1: 0.7370


[I 2021-01-05 17:37:27,599] Trial 1 finished with value: 0.729335771260831 and parameters: {'step': 3, 'no._units': 206, 'number': 6, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 0 with value: 0.7318557327041448.


...

[I 2021-01-05 20:03:37,264] Trial 47 finished with value: 0.7323069785111704 and parameters: {'step': 2, 'no._units': 184, 'number': 3, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 36 with value: 0.7408516980052888.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 22s 111us/sample - loss: 1.1908 - auc_48: 0.7140 - val_loss: 1.3497 - val_auc_48: 0.7318
Epoch 2/50
196038/196038 [==============================] - 21s 105us/sample - loss: 1.1591 - auc_48: 0.7308 - val_loss: 1.4091 - val_auc_48: 0.7317
Epoch 3/50
196038/196038 [==============================] - 21s 105us/sample - loss: 1.1479 - auc_48: 0.7365 - val_loss: 1.2021 - val_auc_48: 0.7382
Epoch 4/50
196038/196038 [==============================] - 21s 106us/sample - loss: 1.1456 - auc_48: 0.7365 - val_loss: 1.1927 - val_auc_48: 0.7381
Epoch 5/50
196038/196038 [==============================] - 21s 105us/sample - loss: 1.1371 - auc_48: 0.7401 - val_loss: 1.2559 - val_auc_48: 0.7365
Epoch 6/50
196038/196038 [==============================] - 21s 106us/sample - loss: 1.1369 - auc_48: 0.7404 - val_loss: 1.2420 - val_auc_48: 0.7381
Epoch

[I 2021-01-05 20:06:25,907] Trial 48 finished with value: 0.7380896708426351 and parameters: {'step': 3, 'no._units': 275, 'number': 2, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 36 with value: 0.7408516980052888.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 22s 112us/sample - loss: 1.4084 - auc_49: 0.7144 - val_loss: 1.5301 - val_auc_49: 0.7280
Epoch 2/50
196038/196038 [==============================] - 21s 105us/sample - loss: 1.3668 - auc_49: 0.7308 - val_loss: 1.4261 - val_auc_49: 0.7370
Epoch 3/50
196038/196038 [==============================] - 21s 106us/sample - loss: 1.3587 - auc_49: 0.7346 - val_loss: 1.8232 - val_auc_49: 0.7336
Epoch 4/50
196038/196038 [==============================] - 21s 105us/sample - loss: 1.3521 - auc_49: 0.7370 - val_loss: 1.7679 - val_auc_49: 0.7365
Epoch 5/50
196038/196038 [==============================] - 21s 106us/sample - loss: 1.3517 - auc_49: 0.7372 - val_loss: 1.9321 - val_auc_49: 0.7373
Epoch 6/50
196038/196038 [==============================] - 21s 105us/sample - loss: 1.3447 - auc_49: 0.7399 - val_loss: 1.8334 - val_auc_49: 0.7357


[I 2021-01-05 20:08:33,431] Trial 49 finished with value: 0.7370289544195239 and parameters: {'step': 4, 'no._units': 221, 'number': 2, 'best_weight': {0: 1.0, 1: 17.0}}. Best is trial 36 with value: 0.7408516980052888.


In [14]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7408516980052888


In [15]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'step': 5, 'no._units': 102, 'number': 2, 'best_weight': {0: 1.0, 1: 12.0}}


In [17]:
# loading the best Neural Network
Best_Model_elu = keras.models.load_model('Best_model_Elu_eq_Learn.h5')

In [18]:
# Printing its summary
Best_Model_elu.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_147 (Bat (None, 103)               412       
_________________________________________________________________
dense_147 (Dense)            (None, 102)               10608     
_________________________________________________________________
batch_normalization_148 (Bat (None, 102)               408       
_________________________________________________________________
dense_148 (Dense)            (None, 102)               10506     
_________________________________________________________________
batch_normalization_149 (Bat (None, 102)               408       
_________________________________________________________________
dense_149 (Dense)            (None, 1)                 103       
Total params: 22,445
Trainable params: 21,831
Non-trainable params: 614
_______________________________________________

In [19]:
# Defining the function to calculate the roc_auc score for the feature sets
def cal_roc_auc(X, y, cls, f_set, t_set, model_name):
    ''' Calculates the roc auc score using the best study parameters 
        f_set : String: specifies 'full feature', 'Reduced feature'
        t_set: String: specifies 'training', 'test'
        model_name: String: specifies Name of the model '''
        
    y_pred = cls.predict_proba(X)
    print('The roc_auc_score for the {} {} set using the best {} classifier is '.format(f_set,t_set,model_name),roc_auc_score(y,y_pred))

In [21]:
# Calculating the Reduced feature training set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_train, y_train, Best_Model_elu, 'Reduced feature', 'training', 'Dense Neural Network using ELU activation')

The roc_auc_score for the Reduced feature training set using the best Dense Neural Network using ELU activation classifier is  0.7455322275775218


In [22]:
# Calculating the Reduced feature test set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_test, y_test, Best_Model_elu, 'Reduced feature', 'test', 'Dense Neural Network using ELU activation')

The roc_auc_score for the Reduced feature test set using the best Dense Neural Network using ELU activation classifier is  0.7453847568202309


### Calculating R_R ratio for Best Dense Neural Network with ELU activation & RMS Optimizer

In [55]:
# Defining the exponential decay learning rate with step 5.
def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /5 )

In [57]:
from sklearn.model_selection import StratifiedKFold
cv_strat = StratifiedKFold(5,random_state=42)

In [59]:
# Computing  5 fold  CV predictions on the training set for the Neural Network with ELU activation & RMS Optimizer
scores_neural = []
for train_index, test_index in cv_strat.split(X_train, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train.loc[train_index], X_train.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    
    # Splitting the Training set further into training & validation set.
    X_tr_r, X_val, y_tr_r, y_val = train_test_split(X_tr, y_tr, test_size=0.15, random_state=42, stratify=y_tr)
    
    # Loading fresh best keras dense model with ELU activation & RMS Optimizer for training on 4 folds
    neural = keras.models.load_model('Best_model_Elu_eq_Learn.h5')
    
    # comiling & Training the Neural Net
    neural.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(),
                   metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
    # defining Checkpoints
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=4)# 2nd Callback
    lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
    
    # Fitting The model
    neural.fit(X_tr_r, y_tr_r, epochs=50, validation_data=(X_val,y_val),batch_size=32,
    class_weight={0: 1.0, 1: 12},callbacks=[checkpoint_cb,early_stopping_cb,lr_scheduler_cb])
    
    # Loading the best Neural model after training & before making predictions
    neural_best = keras.models.load_model('best_model.h5')    
    
    # Saving the predictions for every fold in the scores list
    y_pred = neural_best.predict_proba(X_tst)
    scores_neural.append(roc_auc_score(y_tst,y_pred))
      

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 156830 samples, validate on 27676 samples
Epoch 1/50
156830/156830 [==============================] - 19s 121us/sample - loss: 1.1571 - auc_151: 0.7286 - val_loss: 1.1630 - val_auc_151: 0.7339
Epoch 2/50
156830/156830 [==============================] - 18s 113us/sample - loss: 1.1452 - auc_151: 0.7356 - val_loss: 1.1719 - val_auc_151: 0.7380
Epoch 3/50
156830/156830 [==============================] - 18s 116us/sample - loss: 1.1378 - auc_151: 0.7399 - val_loss: 1.1306 - val_auc_151: 0.7432
Epoch 4/50
156830/156830 [==============================] - 18s 113us/sample - loss: 1.1328 - auc_151: 0.7423 - val_loss: 1.1427 - val_auc_151: 0.7415
Epoch 5/50
156830/156830 [==============================] - 18s 113us/sample - loss: 1.1334 - auc_151: 0.7419 - val_loss: 1.5397 - val_auc_151: 0.7420
Epoch 6/50
156830/156830 [==============================] - 18s 113us/sample - loss: 1.1320 - auc_151: 0.7420 - val_loss: 1.8849 - val_auc_151: 

In [64]:
print('The reward associated with the best Dense Neural Classifier (with ELU activation & RMS Optimizer) using roc_auc metric is: ',np.mean(scores_neural))

The reward associated with the best Dense Neural Classifier (with ELU activation & RMS Optimizer) using roc_auc metric is:  0.7389628876551522


In [65]:
print('The risk associated with the best Dense Neural Classifier (with ELU activation & RMS Optimizer) using roc_auc metric is: ',np.std(scores_neural))

The risk associated with the best Dense Neural Classifier (with ELU activation & RMS Optimizer) using roc_auc metric is:  0.0036465761343255983


In [63]:
R_R_Ratio_elu_rms = np.mean(scores_neural)/np.std(scores_neural)

In [66]:
print('The reward risk ratio for the best Dense Neural Classifier(with ELU activation & RMS Optimizer) using roc_auc metric is: ',R_R_Ratio_elu_rms)

The reward risk ratio for the best Dense Neural Classifier(with ELU activation & RMS Optimizer) using roc_auc metric is:  202.64567650164165


In [91]:
print('5 fold CV roc_auc scores for the best Dense Neural Classifier(with ELU activation & RMS Optimizer) are: ',scores_neural)

5 fold CV roc_auc scores for the best Dense Neural Classifier(with ELU activation & RMS Optimizer) are:  [0.7340358172122319, 0.7392319218339563, 0.7449286066261093, 0.7366216866620908, 0.739996405941373]


#### R_R Ratio for the best Dense Neural Classifier (with ELU activation & RMS Optimizer) classifier using roc_auc metric is:  202.64567650164165

## Observations:
### 1)  The Dense Neural Network(with ELU activation & RMS Optimizer) classifier has fitted the Training set very well, with no overfitting at all . Though it has the highest R_R ratio of all the models fitted till now, its test set roc_auc score is the one of the smallest of the lot.
### 2) ELU activation function has clearly well fitted the dataset . May be trying a better optimizer such as Nadam may result in better test set roc_auc score.

## Model_13: Keras Dense Model with Equal Neurons/layer, ELU Activation, Nadam optimizer & Tuned learning rate.

In [23]:
# Defining best roc_auc score
best_roc_auc = 0

In [24]:
# defining the objective function
def objective_wrappper_keras_eq_Nadam(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        model.add(keras.layers.BatchNormalization())# For normalization
        for i in range(trial.suggest_int('number',2,6)):
            model.add(keras.layers.Dense(units=no_units,activation='elu',
                                     kernel_initializer='he_normal'))
            model.add(keras.layers.BatchNormalization())# For normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Elu_eq_Learn_nadam.h5')
        
        return roc_auc_1
    
    return objective

In [25]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq_Nadam, cl_weight,
                                                            n_trials=50)

[I 2021-01-05 23:19:25,516] A new study created in memory with name: no-name-8eacc11f-b1f0-41c8-9efb-5b40f409705d


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 44s 227us/sample - loss: 1.1541 - auc: 0.7215 - val_loss: 1.1242 - val_auc: 0.7429
Epoch 2/50
196038/196038 [==============================] - 43s 217us/sample - loss: 1.1006 - auc: 0.7441 - val_loss: 1.1796 - val_auc: 0.7509
Epoch 3/50
196038/196038 [==============================] - 42s 214us/sample - loss: 1.0872 - auc: 0.7518 - val_loss: 1.1380 - val_auc: 0.7547
Epoch 4/50
196038/196038 [==============================] - 42s 214us/sample - loss: 1.0730 - auc: 0.7598 - val_loss: 1.1439 - val_auc: 0.7557
Epoch 5/50
196038/196038 [==============================] - 42s 213us/sample - loss: 1.0646 - auc: 0.7647 - val_loss: 1.2415 - val_auc: 0.7567


[I 2021-01-05 23:23:02,880] Trial 0 finished with value: 0.742829778318869 and parameters: {'step': 4, 'no._units': 475, 'number': 6, 'best_weight': {0: 1.0, 1: 11.38697}}. Best is trial 0 with value: 0.742829778318869.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 38s 191us/sample - loss: 1.2881 - auc_1: 0.7298 - val_loss: 1.3115 - val_auc_1: 0.7387
Epoch 2/50
196038/196038 [==============================] - 35s 180us/sample - loss: 1.2516 - auc_1: 0.7474 - val_loss: 1.2611 - val_auc_1: 0.7483
Epoch 3/50
196038/196038 [==============================] - 36s 181us/sample - loss: 1.2399 - auc_1: 0.7534 - val_loss: 3.6865 - val_auc_1: 0.7524
Epoch 4/50
196038/196038 [==============================] - 36s 182us/sample - loss: 1.2297 - auc_1: 0.7581 - val_loss: 9.6753 - val_auc_1: 0.7465
Epoch 5/50
196038/196038 [==============================] - 36s 181us/sample - loss: 1.2215 - auc_1: 0.7620 - val_loss: 4.2302 - val_auc_1: 0.7537
Epoch 6/50
196038/196038 [==============================] - 36s 185us/sample - loss: 1.2196 - auc_1: 0.7629 - val_loss: 17.3467 - val_auc_1: 0.7432


[I 2021-01-05 23:26:42,346] Trial 1 finished with value: 0.7482424369295579 and parameters: {'step': 3, 'no._units': 192, 'number': 5, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 1 with value: 0.7482424369295579.


...

[I 2021-01-06 02:37:04,847] Trial 47 finished with value: 0.7537201462693452 and parameters: {'step': 2, 'no._units': 223, 'number': 6, 'best_weight': {0: 1.0, 1: 14.0}}. Best is trial 39 with value: 0.7590064130471813.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 38s 193us/sample - loss: 1.2517 - auc_48: 0.7286 - val_loss: 1.2642 - val_auc_48: 0.7465
Epoch 2/50
196038/196038 [==============================] - 36s 183us/sample - loss: 1.2065 - auc_48: 0.7501 - val_loss: 1.2053 - val_auc_48: 0.7509
Epoch 3/50
196038/196038 [==============================] - 36s 184us/sample - loss: 1.1953 - auc_48: 0.7555 - val_loss: 1.2031 - val_auc_48: 0.7552
Epoch 4/50
196038/196038 [==============================] - 36s 183us/sample - loss: 1.1836 - auc_48: 0.7617 - val_loss: 1.1989 - val_auc_48: 0.7544
Epoch 5/50
196038/196038 [==============================] - 36s 183us/sample - loss: 1.1831 - auc_48: 0.7619 - val_loss: 1.1994 - val_auc_48: 0.7537
Epoch 6/50
196038/196038 [==============================] - 36s 186us/sample - loss: 1.1786 - auc_48: 0.7645 - val_loss: 1.2049 - val_auc_48: 0.7544
Epoch

[I 2021-01-06 02:41:59,007] Trial 48 finished with value: 0.7544346910381958 and parameters: {'step': 2, 'no._units': 262, 'number': 5, 'best_weight': {0: 1.0, 1: 14.0}}. Best is trial 39 with value: 0.7590064130471813.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 38s 194us/sample - loss: 1.2925 - auc_49: 0.7284 - val_loss: 1.2819 - val_auc_49: 0.7415
Epoch 2/50
196038/196038 [==============================] - 36s 183us/sample - loss: 1.2475 - auc_49: 0.7497 - val_loss: 1.2596 - val_auc_49: 0.7543
Epoch 3/50
196038/196038 [==============================] - 35s 181us/sample - loss: 1.2309 - auc_49: 0.7580 - val_loss: 1.2412 - val_auc_49: 0.7566
Epoch 4/50
196038/196038 [==============================] - 36s 182us/sample - loss: 1.2226 - auc_49: 0.7620 - val_loss: 1.2644 - val_auc_49: 0.7578
Epoch 5/50
196038/196038 [==============================] - 36s 182us/sample - loss: 1.2195 - auc_49: 0.7636 - val_loss: 1.2514 - val_auc_49: 0.7574
Epoch 6/50
196038/196038 [==============================] - 35s 180us/sample - loss: 1.2181 - auc_49: 0.7640 - val_loss: 1.2379 - val_auc_49: 0.7568
Epoch

[I 2021-01-06 02:51:01,192] Trial 49 finished with value: 0.7570586402037869 and parameters: {'step': 2, 'no._units': 185, 'number': 5, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 39 with value: 0.7590064130471813.


In [26]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7590064130471813


In [27]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'step': 3, 'no._units': 237, 'number': 2, 'best_weight': {0: 1.0, 1: 14.0}}


In [28]:
# loading the best Neural Network
Best_Model_elu = keras.models.load_model('Best_model_Elu_eq_Learn_nadam.h5')

In [29]:
# Printing its summary
Best_Model_elu.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_172 (Bat (None, 103)               412       
_________________________________________________________________
dense_172 (Dense)            (None, 237)               24648     
_________________________________________________________________
batch_normalization_173 (Bat (None, 237)               948       
_________________________________________________________________
dense_173 (Dense)            (None, 237)               56406     
_________________________________________________________________
batch_normalization_174 (Bat (None, 237)               948       
_________________________________________________________________
dense_174 (Dense)            (None, 1)                 238       
Total params: 83,600
Trainable params: 82,446
Non-trainable params: 1,154
_____________________________________________

In [32]:
# Calculating the Reduced feature training set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_train, y_train, Best_Model_elu, 'Reduced feature', 'training', 'Dense Neural Network using ELU activation')

The roc_auc_score for the Reduced feature training set using the best Dense Neural Network using ELU activation classifier is  0.7727873525704507


In [33]:
# Calculating the Reduced feature test set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_test, y_test, Best_Model_elu, 'Reduced feature', 'test', 'Dense Neural Network using ELU activation')

The roc_auc_score for the Reduced feature test set using the best Dense Neural Network using ELU activation classifier is  0.7627502861257128


### Calculating R_R ratio for Best Dense Neural Network with ELU activation & Nadam Optimizer

In [67]:
# Defining the exponential decay learning rate with step 3.
def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /3 )

In [68]:
# Computing  5 fold  CV predictions on the training set for the Neural Network with ELU activation & Nadam Optimizer
scores_neural_nadam = []
for train_index, test_index in cv_strat.split(X_train, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train.loc[train_index], X_train.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    
    # Splitting the Training set further into training & validation set.
    X_tr_r, X_val, y_tr_r, y_val = train_test_split(X_tr, y_tr, test_size=0.15, random_state=42, stratify=y_tr)
    
    # Loading fresh best keras dense model with ELU activation & Nadam Optimizer for training on 4 folds
    neural = keras.models.load_model('Best_model_Elu_eq_Learn_nadam.h5')
    
    # comiling & Training the Neural Net
    neural.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                   metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
    # defining Checkpoints
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)# 2nd Callback
    lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
    
    # Fitting The model
    neural.fit(X_tr_r, y_tr_r, epochs=50, validation_data=(X_val,y_val),batch_size=32,
    class_weight={0: 1.0, 1: 14},callbacks=[checkpoint_cb,early_stopping_cb,lr_scheduler_cb])
    
    # Loading the best Neural model after training & before making predictions
    neural_best = keras.models.load_model('best_model.h5')    
    
    # Saving the predictions for every fold in the scores list
    y_pred = neural_best.predict_proba(X_tst)
    scores_neural_nadam.append(roc_auc_score(y_tst,y_pred))
      

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 156830 samples, validate on 27676 samples
Epoch 1/50
156830/156830 [==============================] - 22s 138us/sample - loss: 1.2058 - auc_156: 0.7502 - val_loss: 1.2193 - val_auc_156: 0.7572
Epoch 2/50
156830/156830 [==============================] - 20s 126us/sample - loss: 1.1882 - auc_156: 0.7597 - val_loss: 1.1920 - val_auc_156: 0.7624
Epoch 3/50
156830/156830 [==============================] - 20s 126us/sample - loss: 1.1738 - auc_156: 0.7662 - val_loss: 1.1758 - val_auc_156: 0.7645
Epoch 4/50
156830/156830 [==============================] - 20s 125us/sample - loss: 1.1644 - auc_156: 0.7712 - val_loss: 1.1798 - val_auc_156: 0.7636
Epoch 5/50
156830/156830 [==============================] - 20s 127us/sample - loss: 1.1553 - auc_156: 0.7751 - val_loss: 1.1815 - val_auc_156: 0.7630
Epoch 6/50
156830/156830 [==============================] - 20s 129us/sample - loss: 1.1518 - auc_156: 0.7770 - val_loss: 1.1819 - val_auc_156: 

In [69]:
print('The reward associated with the best Dense Neural Classifier (with ELU activation & Nadam Optimizer) using roc_auc metric is: ',np.mean(scores_neural_nadam))

The reward associated with the best Dense Neural Classifier (with ELU activation & Nadam Optimizer) using roc_auc metric is:  0.7592933689511144


In [70]:
print('The risk associated with the best Dense Neural Classifier (with ELU activation & Nadam Optimizer) using roc_auc metric is: ',np.std(scores_neural_nadam))

The risk associated with the best Dense Neural Classifier (with ELU activation & Nadam Optimizer) using roc_auc metric is:  0.0050777337467766874


In [71]:
R_R_Ratio_elu_nadam = np.mean(scores_neural_nadam)/np.std(scores_neural_nadam)

In [72]:
print('The reward risk ratio for the best Dense Neural Classifier(with ELU activation & Nadam Optimizer) using roc_auc metric is: ',R_R_Ratio_elu_nadam)

The reward risk ratio for the best Dense Neural Classifier(with ELU activation & Nadam Optimizer) using roc_auc metric is:  149.5339076085092


In [92]:
print('5 fold CV roc_auc scores for the best Dense Neural Classifier(with ELU activation & Nadam Optimizer) are: ',scores_neural_nadam)

5 fold CV roc_auc scores for the best Dense Neural Classifier(with ELU activation & Nadam Optimizer) are:  [0.7551102987489511, 0.7560758435889806, 0.76775648030497, 0.7625619551685514, 0.7549622669441196]


#### R_R Ratio for the best Dense Neural Classifier (with ELU activation & Nadam Optimizer) classifier using roc_auc metric is:  149.5339076085092

## Observations:
### 1)  The Dense Neural Classifier (with ELU activation & Nadam Optimizer)  has beautifully fitted the Training set, with no signs of overfitting. But though its test set roc_auc score is better than the previous dense neural net (with ELU activation & RMS Optimizer), its R_R ratio is much less than that of the latter.

### 2) May be trying a different activation function  such as Leaky RELU may result in comparable/better test set roc_auc score & R_R ratio.

## Model_14: Keras Dense Model with Equal Neurons/layer, Leaky RELU Activation, Nadam optimizer & Tuned learning rate.

In [34]:
# Defining best roc_auc score
best_roc_auc = 0

In [35]:
# defining the objective function
def objective_wrappper_keras_eq_leaky(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        model.add(keras.layers.BatchNormalization())# For normalization
        for i in range(trial.suggest_int('number',2,5)):
            model.add(keras.layers.Dense(units=no_units, kernel_initializer='he_normal'))
            model.add(keras.layers.LeakyReLU(alpha=.2))
            model.add(keras.layers.BatchNormalization())# For normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Elu_eq_Learn_leaky.h5')
        
        return roc_auc_1
    
    return objective

In [36]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq_leaky, cl_weight,
                                                            n_trials=50)

[I 2021-01-06 03:18:03,434] A new study created in memory with name: no-name-4a0ddc42-a0c9-442a-9fab-575320ac69b5


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 35s 179us/sample - loss: 1.3941 - auc_50: 0.7254 - val_loss: 1.3443 - val_auc_50: 0.7459
Epoch 2/50
196038/196038 [==============================] - 34s 171us/sample - loss: 1.3347 - auc_50: 0.7441 - val_loss: 1.3254 - val_auc_50: 0.7520
Epoch 3/50
196038/196038 [==============================] - 33s 171us/sample - loss: 1.3221 - auc_50: 0.7498 - val_loss: 1.3325 - val_auc_50: 0.7510
Epoch 4/50
196038/196038 [==============================] - 34s 172us/sample - loss: 1.3144 - auc_50: 0.7530 - val_loss: 1.3253 - val_auc_50: 0.7555
Epoch 5/50
196038/196038 [==============================] - 34s 172us/sample - loss: 1.3056 - auc_50: 0.7571 - val_loss: 1.3144 - val_auc_50: 0.7567
Epoch 6/50
196038/196038 [==============================] - 34s 172us/sample - loss: 1.2968 - auc_50: 0.7611 - val_loss: 1.3109 - val_auc_50: 0.7583
Epoch

[I 2021-01-06 03:25:30,279] Trial 0 finished with value: 0.7578955969798178 and parameters: {'step': 5, 'no._units': 503, 'number': 4, 'best_weight': {0: 1.0, 1: 17.0}}. Best is trial 0 with value: 0.7578955969798178.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.2896 - auc_51: 0.7317 - val_loss: 1.3333 - val_auc_51: 0.7455
Epoch 2/50
196038/196038 [==============================] - 28s 141us/sample - loss: 1.2542 - auc_51: 0.7456 - val_loss: 1.2749 - val_auc_51: 0.7477
Epoch 3/50
196038/196038 [==============================] - 27s 139us/sample - loss: 1.2451 - auc_51: 0.7505 - val_loss: 1.2853 - val_auc_51: 0.7541
Epoch 4/50
196038/196038 [==============================] - 28s 141us/sample - loss: 1.2361 - auc_51: 0.7545 - val_loss: 1.2436 - val_auc_51: 0.7538
Epoch 5/50
196038/196038 [==============================] - 28s 141us/sample - loss: 1.2300 - auc_51: 0.7579 - val_loss: 1.2470 - val_auc_51: 0.7566
Epoch 6/50
196038/196038 [==============================] - 28s 141us/sample - loss: 1.2280 - auc_51: 0.7587 - val_loss: 1.2346 - val_auc_51: 0.7573
Epoch

[I 2021-01-06 03:30:10,432] Trial 1 finished with value: 0.7573424068404996 and parameters: {'step': 5, 'no._units': 301, 'number': 3, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 0 with value: 0.7578955969798178.


...

[I 2021-01-06 08:46:30,977] Trial 47 finished with value: 0.7579011191990326 and parameters: {'step': 4, 'no._units': 435, 'number': 4, 'best_weight': {0: 1.0, 1: 15.0}}. Best is trial 29 with value: 0.759045592094821.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 31s 156us/sample - loss: 1.1650 - auc_98: 0.7292 - val_loss: 1.1509 - val_auc_98: 0.7472
Epoch 2/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1272 - auc_98: 0.7454 - val_loss: 1.1507 - val_auc_98: 0.7507
Epoch 3/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1152 - auc_98: 0.7519 - val_loss: 1.1179 - val_auc_98: 0.7553
Epoch 4/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1103 - auc_98: 0.7548 - val_loss: 1.1248 - val_auc_98: 0.7565
Epoch 5/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1028 - auc_98: 0.7587 - val_loss: 1.1101 - val_auc_98: 0.7571
Epoch 6/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.0986 - auc_98: 0.7612 - val_loss: 1.1076 - val_auc_98: 0.7576
Epoch

[I 2021-01-06 08:51:53,944] Trial 48 finished with value: 0.7576101922536133 and parameters: {'step': 5, 'no._units': 502, 'number': 3, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 29 with value: 0.759045592094821.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 39s 197us/sample - loss: 1.1732 - auc_99: 0.7271 - val_loss: 1.1894 - val_auc_99: 0.7206
Epoch 2/50
196038/196038 [==============================] - 37s 186us/sample - loss: 1.1317 - auc_99: 0.7428 - val_loss: 1.1962 - val_auc_99: 0.7496
Epoch 3/50
196038/196038 [==============================] - 36s 184us/sample - loss: 1.1217 - auc_99: 0.7484 - val_loss: 1.1177 - val_auc_99: 0.7535
Epoch 4/50
196038/196038 [==============================] - 36s 184us/sample - loss: 1.1109 - auc_99: 0.7544 - val_loss: 1.1162 - val_auc_99: 0.7567
Epoch 5/50
196038/196038 [==============================] - 36s 185us/sample - loss: 1.1051 - auc_99: 0.7577 - val_loss: 1.1113 - val_auc_99: 0.7547
Epoch 6/50
196038/196038 [==============================] - 37s 186us/sample - loss: 1.0981 - auc_99: 0.7617 - val_loss: 1.1061 - val_auc_99: 0.7574
Epoch

[I 2021-01-06 08:59:16,420] Trial 49 finished with value: 0.757412079121051 and parameters: {'step': 6, 'no._units': 472, 'number': 5, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 29 with value: 0.759045592094821.


In [37]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.759045592094821


In [38]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'step': 5, 'no._units': 520, 'number': 4, 'best_weight': {0: 1.0, 1: 12.0}}


In [74]:
# loading the best Neural Network
Best_Model_relu_leaky = keras.models.load_model('Best_model_Elu_eq_Learn_leaky.h5')

In [40]:
# Printing its summary
Best_Model_relu_leaky.summary()

Model: "sequential_79"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_380 (Bat (None, 103)               412       
_________________________________________________________________
dense_380 (Dense)            (None, 520)               54080     
_________________________________________________________________
leaky_re_lu_108 (LeakyReLU)  (None, 520)               0         
_________________________________________________________________
batch_normalization_381 (Bat (None, 520)               2080      
_________________________________________________________________
dense_381 (Dense)            (None, 520)               270920    
_________________________________________________________________
leaky_re_lu_109 (LeakyReLU)  (None, 520)               0         
_________________________________________________________________
batch_normalization_382 (Bat (None, 520)             

In [41]:
# Calculating the Reduced feature training set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_train, y_train, Best_Model_relu_leaky, 'Reduced feature', 'training', 'Dense Neural Network using Leaky RELU activation')

The roc_auc_score for the Reduced feature training set using the best Dense Neural Network using Leaky RELU activation classifier is  0.7683335420626638


In [42]:
# Calculating the Reduced feature test set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_test, y_test, Best_Model_relu_leaky, 'Reduced feature', 'test', 'Dense Neural Network using Leaky RELU activation')

The roc_auc_score for the Reduced feature test set using the best Dense Neural Network using Leaky RELU activation classifier is  0.7622685941311187


### Calculating R_R ratio for Best Dense Neural Network with Leaky RELU activation & Nadam Optimizer

In [76]:
# Defining the exponential decay learning rate with step 5.
def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /5 )

In [77]:
# Computing  5 fold  CV predictions on the training set for the Neural Network with Leaky RELU activation 
# & Nadam Optimizer
scores_neural_leaky = []
for train_index, test_index in cv_strat.split(X_train, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train.loc[train_index], X_train.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    
    # Splitting the Training set further into training & validation set.
    X_tr_r, X_val, y_tr_r, y_val = train_test_split(X_tr, y_tr, test_size=0.15, random_state=42, stratify=y_tr)
    
    # Loading fresh best keras dense model with ELU activation & Nadam Optimizer for training on 4 folds
    neural = keras.models.load_model('Best_model_Elu_eq_Learn_leaky.h5')
    
    # comiling & Training the Neural Net
    neural.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                   metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
    # defining Checkpoints
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)# 2nd Callback
    lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
    
    # Fitting The model
    neural.fit(X_tr_r, y_tr_r, epochs=50, validation_data=(X_val,y_val),batch_size=32,
    class_weight={0: 1.0, 1: 12},callbacks=[checkpoint_cb,early_stopping_cb,lr_scheduler_cb])
    
    # Loading the best Neural model after training & before making predictions
    neural_best = keras.models.load_model('best_model.h5')    
    
    # Saving the predictions for every fold in the scores list
    y_pred = neural_best.predict_proba(X_tst)
    scores_neural_leaky.append(roc_auc_score(y_tst,y_pred))
      

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 156830 samples, validate on 27676 samples
Epoch 1/50
156830/156830 [==============================] - 30s 192us/sample - loss: 1.1271 - auc_165: 0.7462 - val_loss: 1.1869 - val_auc_165: 0.7583
Epoch 2/50
156830/156830 [==============================] - 27s 173us/sample - loss: 1.1137 - auc_165: 0.7539 - val_loss: 1.1248 - val_auc_165: 0.7613
Epoch 3/50
156830/156830 [==============================] - 28s 181us/sample - loss: 1.1026 - auc_165: 0.7595 - val_loss: 1.0976 - val_auc_165: 0.7633
Epoch 4/50
156830/156830 [==============================] - 29s 185us/sample - loss: 1.0945 - auc_165: 0.7640 - val_loss: 1.0911 - val_auc_165: 0.7657
Epoch 5/50
156830/156830 [==============================] - 27s 172us/sample - loss: 1.0878 - auc_165: 0.7673 - val_loss: 1.1010 - val_auc_165: 0.7675
Epoch 6/50
156830/156830 [==============================] - 28s 177us/sample - loss: 1.0839 - auc_165: 0.7696 - val_loss: 1.0899 - val_auc_165: 

In [78]:
print('The reward associated with the best Dense Neural Classifier (with Leaky RELU activation & Nadam Optimizer) using roc_auc metric is: ',np.mean(scores_neural_leaky))

The reward associated with the best Dense Neural Classifier (with Leaky RELU activation & Nadam Optimizer) using roc_auc metric is:  0.7630302771099005


In [79]:
print('The risk associated with the best Dense Neural Classifier (with Leaky RELU activation & Nadam Optimizer) using roc_auc metric is: ',np.std(scores_neural_leaky))

The risk associated with the best Dense Neural Classifier (with Leaky RELU activation & Nadam Optimizer) using roc_auc metric is:  0.004759616587197034


In [80]:
R_R_Ratio_Leaky_RELU = np.mean(scores_neural_leaky)/np.std(scores_neural_leaky)

In [81]:
print('The reward risk ratio for the best Dense Neural Classifier(with Leaky RELU activation & Nadam Optimizer) using roc_auc metric is: ',R_R_Ratio_Leaky_RELU)

The reward risk ratio for the best Dense Neural Classifier(with Leaky RELU activation & Nadam Optimizer) using roc_auc metric is:  160.313391453082


In [82]:
print('5 fold CV roc_auc scores for the best Dense Neural Classifier(with Leaky RELU activation & Nadam Optimizer) are: ',scores_neural_leaky)

5 fold CV roc_auc scores for the best Dense Neural Classifier(with Leaky RELU activation & Nadam Optimizer) are:  [0.7570462298319943, 0.7592255989426604, 0.770837388945055, 0.7642094466618157, 0.7638327211679767]


#### R_R Ratio for the best Dense Neural Classifier (with Leaky RELU activation & Nadam Optimizer) classifier using roc_auc metric is:  160.313391453082

## Observations:
### 1)  The Dense Neural Classifier (with Leaky RELU activation & Nadam Optimizer) classifier has  beautifully fitted the Training set, with no signs of overfitting. Though its test set roc_auc score is almost equal to the previous dense neural net (with ELU activation & Nadam Optimizer), its R_R ratio is much higher than that of the latter, which is a significant improvement.

### 2) Unfortunately , one big disadvantage of the above classifier is enormous no. of parameters that need to be tuned, which adds to the computational complexity. Infact, of all the Neural nets tested thus far, this model uses most no. layers as well as neurons/layer.
### 3) May be using a different activation function such as RELU , may remedy this problem.

## Model_15: Keras Dense Model with Equal Neurons/layer, RELU Activation, Nadam optimizer & Tuned learning rate.

In [83]:
# Defining best roc_auc score
best_roc_auc = 0

In [47]:
# defining the objective function
def objective_wrappper_keras_eq_RELU(X_tr, y_tr, X_val, y_val, cl_w):
    '''
    Optimizes Keras ,neural net with equal no. of neurons in each layer , best parameters 
    on the given training set: X_tr, y_tr using validation set: X_val,y_val & class weights: cl_w
    
    '''

    def objective(trial):
        s = trial.suggest_int('step',2,6)
        def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /s )
              
        # building model & searching for the best no. of neural units per layer.
        no_units = trial.suggest_int('no._units',100,600)# No. of Neurons in each layer
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=X_tr.shape[1:]))
        model.add(keras.layers.BatchNormalization())# For normalization
        for i in range(trial.suggest_int('number',2,5)):
            model.add(keras.layers.Dense(units=no_units,activation='relu',kernel_initializer='he_normal'))
            model.add(keras.layers.BatchNormalization())# For normalization
        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
    
        # Searching for the best learning rate.
        #lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) 
    
        # Compiling the model
        model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                      metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
        # Defining the Callbacks
        checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
        early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True) # 2nd Callback, Stop if validation score doen't improve for
        # 4 epochs        
        lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
        #tensorboard_cb = keras.callbacks.TensorBoard(get_run_logdir()) 
        
        #Fitting the model
        model.fit(X_tr, y_tr, epochs=50, validation_data=(X_val,y_val), batch_size=32,
             class_weight=trial.suggest_categorical('best_weight',cl_w),
             callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler_cb])#,tensorboard_cb])
    
        # Loading the best model 
        model_best = keras.models.load_model('best_model.h5')
    
        # Computing the roc_auc_score for the validation set
        roc_auc_1 = roc_auc_score(y_val,model_best.predict_proba(X_val))
        global best_roc_auc
        # Updating best_roc_auc score
        if roc_auc_1 > best_roc_auc:
            best_roc_auc = roc_auc_1
            model_best.save('Best_model_Elu_eq_Learn_relu.h5')
        
        return roc_auc_1
    
    return objective

In [48]:
# Extracting the best model parameters and best study score
best_study_score,best_study_params = study_best_score_params(X_train_1, y_train_1, X_val, y_val, objective_wrappper_keras_eq_RELU, cl_weight,
                                                            n_trials=50)

[I 2021-01-06 12:39:20,417] A new study created in memory with name: no-name-e3f71846-5411-489f-876f-226ec55cc28e


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 40s 203us/sample - loss: 1.4402 - auc_100: 0.7221 - val_loss: 1.5305 - val_auc_100: 0.7370
Epoch 2/50
196038/196038 [==============================] - 38s 193us/sample - loss: 1.3703 - auc_100: 0.7445 - val_loss: 1.4534 - val_auc_100: 0.7463
Epoch 3/50
196038/196038 [==============================] - 39s 196us/sample - loss: 1.3553 - auc_100: 0.7515 - val_loss: 792.1159 - val_auc_100: 0.7394
Epoch 4/50
196038/196038 [==============================] - 38s 192us/sample - loss: 1.3378 - auc_100: 0.7594 - val_loss: 1.6402 - val_auc_100: 0.7516
Epoch 5/50
196038/196038 [==============================] - 38s 192us/sample - loss: 1.3274 - auc_100: 0.7634 - val_loss: 72.2401 - val_auc_100: 0.7542
Epoch 6/50
196038/196038 [==============================] - 38s 192us/sample - loss: 1.3185 - auc_100: 0.7675 - val_loss: 12.9367 - val_auc_1

[I 2021-01-06 12:43:13,753] Trial 0 finished with value: 0.7462634672065629 and parameters: {'step': 4, 'no._units': 492, 'number': 5, 'best_weight': {0: 1.0, 1: 18.0}}. Best is trial 0 with value: 0.7462634672065629.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 31s 159us/sample - loss: 1.2179 - auc_101: 0.7276 - val_loss: 1.3312 - val_auc_101: 0.7367
Epoch 2/50
196038/196038 [==============================] - 30s 151us/sample - loss: 1.1693 - auc_101: 0.7477 - val_loss: 1.2179 - val_auc_101: 0.7550
Epoch 3/50
196038/196038 [==============================] - 29s 149us/sample - loss: 1.1512 - auc_101: 0.7570 - val_loss: 1.1625 - val_auc_101: 0.7574
Epoch 4/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1353 - auc_101: 0.7650 - val_loss: 1.1559 - val_auc_101: 0.7578
Epoch 5/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1275 - auc_101: 0.7688 - val_loss: 1.1662 - val_auc_101: 0.7560
Epoch 6/50
196038/196038 [==============================] - 29s 148us/sample - loss: 1.1239 - auc_101: 0.7705 - val_loss: 1.1644 - val_auc_101: 

[I 2021-01-06 12:47:14,569] Trial 1 finished with value: 0.7577180978806545 and parameters: {'step': 3, 'no._units': 433, 'number': 3, 'best_weight': {0: 1.0, 1: 13.0}}. Best is trial 1 with value: 0.7577180978806545.


...

[I 2021-01-06 15:42:50,513] Trial 47 finished with value: 0.7539657752555178 and parameters: {'step': 5, 'no._units': 346, 'number': 3, 'best_weight': {0: 1.0, 1: 14.0}}. Best is trial 27 with value: 0.7580174122946058.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 29s 149us/sample - loss: 1.3716 - auc_148: 0.7289 - val_loss: 1.3456 - val_auc_148: 0.7460
Epoch 2/50
196038/196038 [==============================] - 27s 140us/sample - loss: 1.3275 - auc_148: 0.7476 - val_loss: 141.2668 - val_auc_148: 0.7520
Epoch 3/50
196038/196038 [==============================] - 27s 140us/sample - loss: 1.3101 - auc_148: 0.7551 - val_loss: 64.1289 - val_auc_148: 0.7538
Epoch 4/50
196038/196038 [==============================] - 27s 140us/sample - loss: 1.3004 - auc_148: 0.7594 - val_loss: 3.2749 - val_auc_148: 0.7547
Epoch 5/50
196038/196038 [==============================] - 27s 140us/sample - loss: 1.2901 - auc_148: 0.7644 - val_loss: 26.4487 - val_auc_148: 0.7526


[I 2021-01-06 15:45:12,984] Trial 48 finished with value: 0.7459834158243678 and parameters: {'step': 4, 'no._units': 231, 'number': 3, 'best_weight': {0: 1.0, 1: 17.0}}. Best is trial 27 with value: 0.7580174122946058.


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 196038 samples, validate on 34595 samples
Epoch 1/50
196038/196038 [==============================] - 34s 175us/sample - loss: 1.1755 - auc_149: 0.7257 - val_loss: 1.1527 - val_auc_149: 0.7369
Epoch 2/50
196038/196038 [==============================] - 32s 164us/sample - loss: 1.1225 - auc_149: 0.7485 - val_loss: 1.1916 - val_auc_149: 0.7508
Epoch 3/50
196038/196038 [==============================] - 32s 164us/sample - loss: 1.1086 - auc_149: 0.7559 - val_loss: 2.3621 - val_auc_149: 0.7529
Epoch 4/50
196038/196038 [==============================] - 32s 164us/sample - loss: 1.0954 - auc_149: 0.7633 - val_loss: 2.2866 - val_auc_149: 0.7543
Epoch 5/50
196038/196038 [==============================] - 32s 165us/sample - loss: 1.0863 - auc_149: 0.7680 - val_loss: 2.1950 - val_auc_149: 0.7503


[I 2021-01-06 15:48:00,130] Trial 49 finished with value: 0.7368342131618947 and parameters: {'step': 3, 'no._units': 481, 'number': 4, 'best_weight': {0: 1.0, 1: 12.0}}. Best is trial 27 with value: 0.7580174122946058.


In [49]:
print('The best roc_auc_score for the study is: ',best_study_score)

The best roc_auc_score for the study is:  0.7580174122946058


In [50]:
print('The best study parameters for the classifier are: ',best_study_params)

The best study parameters for the classifier are:  {'step': 2, 'no._units': 175, 'number': 3, 'best_weight': {0: 1.0, 1: 12.0}}


In [51]:
# loading the best Neural Network
Best_Model_Relu = keras.models.load_model('Best_model_Elu_eq_Learn_relu.h5')

In [52]:
# Printing its summary
Best_Model_Relu.summary()

Model: "sequential_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_597 (Bat (None, 103)               412       
_________________________________________________________________
dense_596 (Dense)            (None, 175)               18200     
_________________________________________________________________
batch_normalization_598 (Bat (None, 175)               700       
_________________________________________________________________
dense_597 (Dense)            (None, 175)               30800     
_________________________________________________________________
batch_normalization_599 (Bat (None, 175)               700       
_________________________________________________________________
dense_598 (Dense)            (None, 175)               30800     
_________________________________________________________________
batch_normalization_600 (Bat (None, 175)            

In [53]:
# Calculating the Reduced feature training set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_train, y_train, Best_Model_Relu, 'Reduced feature', 'training', 'Dense Neural Network using RELU activation')

The roc_auc_score for the Reduced feature training set using the best Dense Neural Network using RELU activation classifier is  0.7705065237291805


In [54]:
# Calculating the Reduced feature test set roc_auc score using the pretrained Neural Network having 
# best study parameters & weights
cal_roc_auc(X_test, y_test, Best_Model_Relu, 'Reduced feature', 'test', 'Dense Neural Network using RELU activation')

The roc_auc_score for the Reduced feature test set using the best Dense Neural Network using RELU activation classifier is  0.7625576360042436


### Calculating R_R ratio for Best Dense Neural Network with  RELU activation & Nadam Optimizer

In [84]:
# Defining the exponential decay learning rate with step 2.
def exponential_decay_fn(epoch):
            return 0.01 * 0.1**(epoch /2 )

In [85]:
# Computing  5 fold  CV predictions on the training set for the Neural Network with RELU activation 
# & Nadam Optimizer
scores_neural_relu = []
for train_index, test_index in cv_strat.split(X_train, y_train):
    # Creating  Folds
    X_tr, X_tst = X_train.loc[train_index], X_train.loc[test_index]
    y_tr, y_tst = y_train.loc[train_index], y_train.loc[test_index]
    
    # Splitting the Training set further into training & validation set.
    X_tr_r, X_val, y_tr_r, y_val = train_test_split(X_tr, y_tr, test_size=0.15, random_state=42, stratify=y_tr)
    
    # Loading fresh best keras dense model with ELU activation & Nadam Optimizer for training on 4 folds
    neural = keras.models.load_model('Best_model_Elu_eq_Learn_relu.h5')
    
    # comiling & Training the Neural Net
    neural.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Nadam(beta_1=0.9, beta_2=0.999),
                   metrics=[tf.keras.metrics.AUC(curve="ROC")])
    
    # defining Checkpoints
    checkpoint_cb = keras.callbacks.ModelCheckpoint('best_model.h5',save_best_only=True) # 1st Callback
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=4,restore_best_weights=True)# 2nd Callback
    lr_scheduler_cb = keras.callbacks.LearningRateScheduler(exponential_decay_fn)# 3rd Callback
    
    # Fitting The model
    neural.fit(X_tr_r, y_tr_r, epochs=50, validation_data=(X_val,y_val),batch_size=32,
    class_weight={0: 1.0, 1: 12},callbacks=[checkpoint_cb,early_stopping_cb,lr_scheduler_cb])
    
    # Loading the best Neural model after training & before making predictions
    neural_best = keras.models.load_model('best_model.h5')    
    
    # Saving the predictions for every fold in the scores list
    y_pred = neural_best.predict_proba(X_tst)
    scores_neural_relu.append(roc_auc_score(y_tst,y_pred))
      

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 156830 samples, validate on 27676 samples
Epoch 1/50
156830/156830 [==============================] - 24s 154us/sample - loss: 1.1246 - auc_170: 0.7472 - val_loss: 1.2505 - val_auc_170: 0.7435
Epoch 2/50
156830/156830 [==============================] - 23s 144us/sample - loss: 1.1041 - auc_170: 0.7585 - val_loss: 1.1837 - val_auc_170: 0.7590
Epoch 3/50
156830/156830 [==============================] - 23s 148us/sample - loss: 1.0930 - auc_170: 0.7643 - val_loss: 1.1052 - val_auc_170: 0.7639
Epoch 4/50
156830/156830 [==============================] - 23s 146us/sample - loss: 1.0854 - auc_170: 0.7688 - val_loss: 1.0965 - val_auc_170: 0.7642
Epoch 5/50
156830/156830 [==============================] - 23s 145us/sample - loss: 1.0842 - auc_170: 0.7690 - val_loss: 1.2460 - val_auc_170: 0.7658
Epoch 6/50
156830/156830 [==============================] - 22s 142us/sample - loss: 1.0843 - auc_170: 0.7691 - val_loss: 1.0979 - val_auc_170: 

In [86]:
print('The reward associated with the best Dense Neural Classifier (with RELU activation & Nadam Optimizer) using roc_auc metric is: ',np.mean(scores_neural_relu))

The reward associated with the best Dense Neural Classifier (with RELU activation & Nadam Optimizer) using roc_auc metric is:  0.7605541394822921


In [87]:
print('The risk associated with the best Dense Neural Classifier (with RELU activation & Nadam Optimizer) using roc_auc metric is: ',np.std(scores_neural_relu))

The risk associated with the best Dense Neural Classifier (with RELU activation & Nadam Optimizer) using roc_auc metric is:  0.004307362083129947


In [88]:
R_R_Ratio_RELU = np.mean(scores_neural_relu)/np.std(scores_neural_relu)

In [89]:
print('The reward risk ratio for the best Dense Neural Classifier(with RELU activation & Nadam Optimizer) using roc_auc metric is: ',R_R_Ratio_RELU)

The reward risk ratio for the best Dense Neural Classifier(with RELU activation & Nadam Optimizer) using roc_auc metric is:  176.57074673639582


In [90]:
print('5 fold CV roc_auc scores for the best Dense Neural Classifier(with RELU activation & Nadam Optimizer) are: ',scores_neural_relu)

5 fold CV roc_auc scores for the best Dense Neural Classifier(with RELU activation & Nadam Optimizer) are:  [0.754474342945305, 0.7561972016348781, 0.7645279041242877, 0.7640541507608882, 0.7635170979461023]


#### R_R Ratio for the best Dense Neural Classifier (with RELU activation & Nadam Optimizer) classifier using roc_auc metric is:  176.57074673639582

## Observations:
### 1)  The Dense Neural Classifier (with RELU activation & Nadam Optimizer) classifier has fitted the Training set very well, with no signs of overfitting. Also it uses fewer no. of parameters that need to be tuned as compared to the one using Leaky RELU & Nadam optimizer.

### 2) One big advantage of this model is that both its test set roc_auc score and R_R ratio are better than corresponding values  of the previous dense neural net (with Leaky RELU activation & Nadam Optimizer),  which is a significant improvement.
### 3) Infact this above classifier has one of the highest test set roc_auc score as well as the R_R ratio of all the aforetested Neural Nets.

### Best R_R Ratio for the Neural Net family using roc_auc metric is:  176.57074673639582 ,corresponding to tuned Dense Neural Classifier (with RELU activation & Nadam Optimizer).

### Taking everything into consideration, Dense Neural Classifier (with RELU activation & Nadam Optimizer)., dominates all other Neural nets & is clearly the winner among all other Neural Classifiers for this dataset.